In [ ]:
import pandas as pd

In [ ]:
import pandas as pd

# 단일 JSON 객체인 경우 'lines=True'를 가정하지 않고 파일을 읽어 보세요.
df = pd.read_json('train_original.json')

In [ ]:
import pandas as pd

# 데이터 로드 (예: CSV 파일)


# 1. 데이터프레임의 열 이름 출력
print("데이터프레임의 열 이름:")
print(df.columns)

# 2. 각 열의 데이터 샘플 출력
print("\n각 열의 첫 번째 데이터 샘플:")
for column in df.columns:
    print(f"{column}: {df[column].iloc[0]}")

# 3. 원문 및 요약본으로 추정되는 열 찾기
possible_content_col = None
possible_summary_col = None

for column in df.columns:
    # 데이터 길이를 기준으로 원문과 요약본 추정
    sample = df[column].iloc[0]
    if isinstance(sample, str):  # 문자열 열만 대상으로 처리
        if len(sample) > 100:  # 100자 이상이면 원문일 가능성이 높음
            possible_content_col = column
        elif len(sample) <= 100:  # 100자 이하이면 요약본일 가능성이 높음
            possible_summary_col = column

# 4. 추정 결과 출력
print("\n원문으로 추정되는 열:", possible_content_col)
print("요약본으로 추정되는 열:", possible_summary_col)

# 5. 데이터 샘플 출력
if possible_content_col:
    print(f"\n원문 예시 ({possible_content_col}):")
    print(df[possible_content_col].iloc[0])

if possible_summary_col:
    print(f"\n요약본 예시 ({possible_summary_col}):")
    print(df[possible_summary_col].iloc[0])


데이터프레임의 열 이름:
Index(['name', 'delivery_date', 'documents'], dtype='object')

각 열의 첫 번째 데이터 샘플:
name: 문서요약 프로젝트
delivery_date: 2020-12-23 12:01:15
documents: {'id': '290741778', 'category': '종합', 'media_type': 'online', 'media_sub_type': '지역지', 'media_name': '광양신문', 'size': 'small', 'char_count': '927', 'publish_date': '2018-01-05 18:54:55', 'title': '논 타작물 재배, 2월 말까지 신청하세요', 'text': [[{'index': 0, 'sentence': 'ha당 조사료 400만원…작물별 차등 지원', 'highlight_indices': ''}], [{'index': 1, 'sentence': '이성훈 sinawi@hanmail.net', 'highlight_indices': ''}], [{'index': 2, 'sentence': '전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다.', 'highlight_indices': ''}], [{'index': 3, 'sentence': '쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다.', 'highlight_indices': '35,37'}], [{'index': 4, 'sentence': '올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다.', 'highlight_indices': '9,11;33,34'}, {'index': 5, 'sentence': '지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 

In [ ]:
# 원문과 요약문 추출
documents = df["documents"]
rows = []

for doc in documents:
    # 원문 재구성 (text의 모든 문장을 합침)
    content = " ".join(
        [sentence["sentence"] for paragraph in doc["text"] for sentence in paragraph]
    )
    # 요약문 (abstractive의 첫 번째 요약 사용)
    summary = doc["abstractive"][0] if doc["abstractive"] else None
    rows.append({"content": content, "summary": summary})

# DataFrame 생성
df = pd.DataFrame(rows)

# 데이터 확인
print("데이터 샘플:")
print(df.head())

데이터 샘플:
                                             content  \
0  이명박 대통령이 어제 30대 그룹 총수를 모아놓고 "시대적 요구는 역시 총수가 앞장...   
1  이명박 정부의 첫 대통령실장을 지낸 류우익 주중대사가 통일부 장관에 내정되면서 대북...   
2  한국은행법 개정안이 우여곡절 끝에 국회 본회의를 통과했다. 이 법안은 글로벌 금융위...   
3  이명박 대통령이 어제 30대 그룹 총수를 모아놓고 "시대적 요구는 역시 총수가 앞장...   
4  한나라당과 정부가 비정규직 대책을 이르면 다음주 중 내놓을 예정이다. 비정규직 임금...   

                                             summary  
0  이명박 대통령은 어제 30대 그룹 총수를 모아놓고 시대적 요구는 역시 총수가 앞장서...  
1  류우억 주중대사가 통일부 장관에 내정되면서  대북정책에 대하여  일관성 있게 지켜야...  
2  한국은행은 금융 위기 이후 금융 시스템 안정을 위해 권한이 더욱 부여된 개정안을 통...  
3  30대 그룹 총수들을 모아 회동을 가진 이명박 대통령은 간접적으로 기부에 대해 언급...  
4  내년 총선ㆍ대선을 앞두고 한나라당의 비정규직을 대상으로 한 포퓰리즘 대책들은 정치권...  


In [ ]:
# 변환한 데이터를 CSV로 저장
df.to_csv("text_summarization_data.csv", index=False)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset
import pandas as pd

# 1. 데이터 로드
file_path = "text_summarization_data.csv"
df = pd.read_csv(file_path)

# Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1)  # Train/Test 분리
train_dataset = dataset["train"]
val_dataset = dataset["test"]

# 2. 모델 및 토크나이저 로드
model_name = "gogamza/kobart-base-v2"  # KoBART 모델
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def preprocess_function(examples):
    inputs = [str(input_text) for input_text in examples["content"]]
    targets = [str(target_text) for target_text in examples["summary"]]

    # 입력 텍스트 토큰화
    model_inputs = tokenizer(
        inputs, max_length=512, padding="max_length", truncation=True
    )

    # 출력 텍스트(요약문) 토큰화
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets, max_length=128, padding="max_length", truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 데이터 전처리
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)

# 4. 학습 설정
training_args = Seq2SeqTrainingArguments(
    output_dir="./kobart_results",      # 출력 디렉토리
    evaluation_strategy="epoch",       # 매 에포크마다 평가
    learning_rate=5e-5,                 # 학습률
    per_device_train_batch_size=8,     # 학습 배치 크기
    per_device_eval_batch_size=8,      # 평가 배치 크기
    num_train_epochs=3,                 # 학습 에포크 수
    weight_decay=0.01,                  # 가중치 감소 (정규화)
    save_total_limit=2,                 # 저장할 체크포인트 개수 제한
    predict_with_generate=True,         # 생성 기반 평가 활성화
    logging_dir='./logs',               # 로깅 디렉토리
    logging_steps=10,                   # 로깅 간격
    save_strategy="epoch",              # 체크포인트 저장 전략
    report_to=[]                        # W&B 등 외부 로깅 비활성화
)

# 5. Trainer 초기화
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
)

# 6. 학습 시작
trainer.train()

# 7. 학습된 모델 저장
model.save_pretrained("./fine_tuned_kobart")
tokenizer.save_pretrained("./fine_tuned_kobart")



You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Map:   0%|          | 0/51084 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/5676 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-bfce7249cf0a>:60: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,0.729700,0.776592
2,0.696800,0.766242
3,0.536400,0.775906


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'forced_eos_token_id': 1}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('./fine_tuned_kobart/tokenizer_config.json',
 './fine_tuned_kobart/special_tokens_map.json',
 './fine_tuned_kobart/tokenizer.json')

In [ ]:
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

# 모델을 저장할 경로 지정
model_save_path = '/content/drive/MyDrive/fine_tuned_kobart'
tokenizer_save_path = '/content/drive/MyDrive/fine_tuned_kobart_tokenizer'

# 학습된 모델 저장
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

print(f"Model saved to {model_save_path}")

Mounted at /content/drive
Model saved to /content/drive/MyDrive/fine_tuned_kobart


In [ ]:
import torch

# Save the model's state_dict to a file
torch.save(model.state_dict(), './fine_tuned_kobart/pytorch_model.bin')

In [ ]:
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

# 모델을 저장할 경로 지정
model_save_path = '/content/drive/MyDrive/fine_tuned_kobart'
tokenizer_save_path = '/content/drive/MyDrive/fine_tuned_kobart_tokenizer'

# 학습된 모델 저장
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

print(f"Model saved to {model_save_path}")